In [1]:
import cv2
import glob
import numpy as np
from tqdm import tqdm
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime

from scipy.sparse import lil_matrix
from scipy.optimize import least_squares

import matplotlib.animation as animation
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

import pickle
import json

%matplotlib widget


# Calibrate Camera

In [2]:

def calibrate_camera(images_folder, row = 5, col = 7, scale= 4.5,show=False, error_threshold = 1.5):
    images_names = sorted(glob.glob(images_folder))
    
    #criteria used by checkerboard pattern detector.
    #Change this if the code can't find the checkerboard
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
    rows = row #number of checkerboard rows.
    columns = col #number of checkerboard columns.
    world_scaling = scale # real world square size.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp

        
    ideal = False
    unideal_images = {}
    
    while not ideal:
        
        i=0
        get = []
        
        images = []
        
        for imname in images_names:
            if imname not in unideal_images.keys():
                im = cv2.imread(imname, 1)
                images.append(im)
        
        #frame dimensions. Frames should be the same size.
        width = images[0].shape[1]
        height = images[0].shape[0]
        
        #Pixel coordinates of checkerboards
        imgpoints = [] # 2d points in image plane.
    
        #coordinates of the checkerboard in checkerboard world space.
        objpoints = [] # 3d point in real world space
            
        for frame in images:
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # detectChessboardflags = (cv2.CALIB_CB_ACCURACY  
            # + cv2.CALIB_CB_EXHAUSTIVE  
            # + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
            # ret, corners = cv2.findChessboardCornersSB(gray, (rows, columns), detectChessboardflags)

            flags = (cv2.CALIB_CB_ADAPTIVE_THRESH  
            + cv2.CALIB_CB_FAST_CHECK  
            + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
            #find the checkerboard
            ret, corners = cv2.findChessboardCorners(gray, (rows, columns), flags)
    
    
            if ret == True:
                get.append(images_names[i])
                #Convolution size used to improve corner detection. Don't make this too large.
                conv_size = (11, 11)
    
                #opencv can attempt to improve the checkerboard coordinates
                corners = cv2.cornerSubPix(gray, corners, conv_size, (-1, -1), criteria)
                cv2.drawChessboardCorners(frame, (rows,columns), corners, ret)
                
                if show:
                    cv2.imshow('img', frame)
                    
                    if cv2.waitKey(0) & 0xFF == ord('q'):
                        show = False
                
                    
    
                objpoints.append(objp)
                imgpoints.append(corners)
            i += 1
    
        cv2.destroyAllWindows()

        
   
        yes =  cv2.calibrateCameraExtended(
            objpoints, 
            imgpoints, 
            (width, height), 
            None, 
            None,
            flags=cv2.CALIB_RATIONAL_MODEL
            )
        
        unideal_detected = False
        for j , error in enumerate(yes[-1]):
            if error >= error_threshold:
                unideal_detected = True
                unideal_images[get[j]] = error
        
        if not unideal_detected:
            ideal = True
    
    print('Total Images : {n}'.format(n=len(images_names)))   
    print('Detected Images : {n}'.format(n=len(get)))        
    print("RMSE :", yes[0])
    e = 0
    for b in get:
        print(b,":",yes[-1][e])
        e +=1
    print("Unideal Images:")
    print(unideal_images)
        
    return yes
 


# Do Calibrate

In [3]:
yes1 = calibrate_camera(images_folder = './data/k2/m/mono/*')
mtx_m,dist_m = yes1[1],yes1[2]
print("Camera Matrix", mtx_m)
print("Distortion Matrix", dist_m)

Total Images : 12
Detected Images : 12
RMSE : 0.3188824467580522
./data/k2/m/mono\IMG_20230607_094327.jpg : [0.20524449]
./data/k2/m/mono\IMG_20230607_094331.jpg : [0.27293389]
./data/k2/m/mono\IMG_20230607_094335.jpg : [0.28384838]
./data/k2/m/mono\IMG_20230607_094339.jpg : [0.2854551]
./data/k2/m/mono\IMG_20230607_094343.jpg : [0.28295238]
./data/k2/m/mono\IMG_20230607_094347.jpg : [0.25115188]
./data/k2/m/mono\IMG_20230607_094352.jpg : [0.38606883]
./data/k2/m/mono\IMG_20230607_094356.jpg : [0.32903022]
./data/k2/m/mono\IMG_20230607_094410.jpg : [0.2408523]
./data/k2/m/mono\IMG_20230607_094414.jpg : [0.44509648]
./data/k2/m/mono\IMG_20230607_094420.jpg : [0.36061386]
./data/k2/m/mono\IMG_20230607_094424.jpg : [0.39363314]
Unideal Images:
{}
Camera Matrix [[1.43685473e+03 0.00000000e+00 9.51710378e+02]
 [0.00000000e+00 1.43760505e+03 5.48900808e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Matrix [[ 6.37019460e+00 -4.65764597e+01  2.20637978e-03 -1.16933464e-03
   

In [4]:
yes1 = calibrate_camera(images_folder = './data/k/j/mono/*')
mtx_j,dist_j = yes1[1],yes1[2]

print("Camera Matrix", mtx_j)
print("Distortion Matrix", dist_j)

Total Images : 20
Detected Images : 18
RMSE : 0.20179337303308906
./data/k/j/mono\IMG_20230529_105904.jpg : [0.13881654]
./data/k/j/mono\IMG_20230529_105916.jpg : [0.20513747]
./data/k/j/mono\IMG_20230529_105928.jpg : [0.17312852]
./data/k/j/mono\IMG_20230529_105933.jpg : [0.25186026]
./data/k/j/mono\IMG_20230529_105947.jpg : [0.18879454]
./data/k/j/mono\IMG_20230529_105952.jpg : [0.25460425]
./data/k/j/mono\IMG_20230529_110006.jpg : [0.10470776]
./data/k/j/mono\IMG_20230529_110018.jpg : [0.17455415]
./data/k/j/mono\IMG_20230529_110030.jpg : [0.14259216]
./data/k/j/mono\IMG_20230529_110045.jpg : [0.17499664]
./data/k/j/mono\IMG_20230529_110052.jpg : [0.19241862]
./data/k/j/mono\IMG_20230529_110107.jpg : [0.13318791]
./data/k/j/mono\IMG_20230529_110116.jpg : [0.09927887]
./data/k/j/mono\IMG_20230529_110129.jpg : [0.22913217]
./data/k/j/mono\IMG_20230529_110136.jpg : [0.17470888]
./data/k/j/mono\IMG_20230529_110143.jpg : [0.31337315]
./data/k/j/mono\IMG_20230529_110149.jpg : [0.26240383]

In [5]:
# mtx, dist = calibrate_camera(images_folder = './data/1/*')
yes1 = calibrate_camera(images_folder = './data/k2/w/mono/*')
mtx_w,dist_w = yes1[1],yes1[2]

print("Camera Matrix", mtx_w)
print("Distortion Matrix", dist_w)

Total Images : 10
Detected Images : 9
RMSE : 0.4727283805356513
./data/k2/w/mono\IMG_20230607_094449.jpg : [0.28357556]
./data/k2/w/mono\IMG_20230607_094455.jpg : [0.62535427]
./data/k2/w/mono\IMG_20230607_094459.jpg : [0.37168421]
./data/k2/w/mono\IMG_20230607_094504.jpg : [0.40268635]
./data/k2/w/mono\IMG_20230607_094509.jpg : [0.40709992]
./data/k2/w/mono\IMG_20230607_094513.jpg : [0.77423412]
./data/k2/w/mono\IMG_20230607_094519.jpg : [0.34377641]
./data/k2/w/mono\IMG_20230607_094531.jpg : [0.35805278]
./data/k2/w/mono\IMG_20230607_094543.jpg : [0.47739685]
Unideal Images:
{}
Camera Matrix [[1.34638176e+03 0.00000000e+00 9.83342855e+02]
 [0.00000000e+00 1.34638338e+03 5.30830400e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Matrix [[-1.76354285e+01 -2.63732159e+01  3.62222957e-03  8.46199027e-04
   1.35311337e+03 -1.79418002e+01 -1.86496552e+01  1.30100598e+03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


# Stereo Prep

In [6]:
def get_images_by_date_taken(folder_name):
    files = sorted(glob.glob(folder_name))
    file_date = {}
    for im in files:
        image = Image.open(im)
        exifdata = image.getexif()

        meta = {}
        # iterating over all EXIF data fields
        for tag_id in exifdata:
            # # get the tag name, instead of human unreadable tag id
            tag = TAGS.get(tag_id, tag_id)
            data = exifdata.get(tag_id)
            # decode bytes 
            if isinstance(data, bytes):
                data = data.decode()
            meta[tag] = data
        file_date[im] = datetime.strptime(meta["DateTime"], "%Y:%m:%d %H:%M:%S")
    return [im for im, _ in sorted(file_date.items(), key=lambda x:x[1])]


In [7]:
def check_image_pair(folder1, folder2):
    c1_images_names = get_images_by_date_taken(folder1)
    c2_images_names = get_images_by_date_taken(folder2)
 
    c1_images = []
    c2_images = []
    for im1, im2 in zip(c1_images_names, c2_images_names):
        _im = cv2.imread(im1, 1)
        c1_images.append(_im)
 
        _im = cv2.imread(im2, 1)
        c2_images.append(_im)
      
    width = c1_images[0].shape[1]
    height = c1_images[0].shape[0]  
    
    i = 0

    for frame1, frame2 in zip(c1_images, c2_images):
        Sres = np.concatenate((frame1, frame2), axis=1)
        
        text = str(i+1) + ' | Camera 1 : ' + str(c1_images_names[i])
        cv2.putText(Sres, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (255,255,0), 2, cv2.LINE_AA)
        
        text = str(i+1) + ' | Camera 2 : ' + str(c2_images_names[i])
        cv2.putText(Sres, text, (width+50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (255,255,0), 2, cv2.LINE_AA)
        

        cv2.imshow('Sres', Sres)
        if cv2.waitKey(0) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
        i += 1
    cv2.destroyAllWindows()


# Stere Calibrate

In [8]:

def stereo_calibrate(mtx1, dist1, mtx2, dist2, folder1, folder2, show=False, error_threshold = 1.5):
    #read the synched frames
    c1_images_names = get_images_by_date_taken(folder1)
    c2_images_names = get_images_by_date_taken(folder2)
 
    #change this if stereo calibration not good.
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
 
    rows = 5 #number of checkerboard rows.
    columns = 7 #number of checkerboard columns.
    world_scaling = 4.5 # real world square size.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 

    ideal = False
    unideal_images = {}
    
    while not ideal:
        
        get = []
        i = 0
        
        
        c1_images = []
        c2_images = []

        for im1, im2 in zip(c1_images_names, c2_images_names):
            if tuple([im1,im2]) not in unideal_images.keys():

                _im = cv2.imread(im1, 1)
                c1_images.append(_im)
        
                _im = cv2.imread(im2, 1)
                c2_images.append(_im)

        if c1_images == [] or c2_images == []:
            print("Calibration Failed! Error Threshold Too Low!")
            break
        
        #frame dimensions. Frames should be the same size.
        width = c1_images[0].shape[1]
        height = c1_images[0].shape[0]
    
        #Pixel coordinates of checkerboards
        imgpoints_left = [] # 2d points in image plane.
        imgpoints_right = []
    
        #coordinates of the checkerboard in checkerboard world space.
        objpoints = [] # 3d point in real world space
    
   
    
        for frame1, frame2 in zip(c1_images, c2_images):
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
                      
            # detectChessboardflags = (cv2.CALIB_CB_ACCURACY  
            # + cv2.CALIB_CB_EXHAUSTIVE  
            # + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
            # c_ret1, corners1 = cv2.findChessboardCornersSB(gray1, (rows, columns), detectChessboardflags)
            # c_ret2, corners2 = cv2.findChessboardCornersSB(gray2, (rows, columns), detectChessboardflags)
            
            flags = (cv2.CALIB_CB_ADAPTIVE_THRESH  
            + cv2.CALIB_CB_FAST_CHECK  
            + cv2.CALIB_CB_NORMALIZE_IMAGE)
                #find the checkerboard

            c_ret1, corners1 = cv2.findChessboardCorners(gray1, (rows, columns), flags)
            c_ret2, corners2 = cv2.findChessboardCorners(gray2, (rows, columns), flags)
    
    
            if c_ret1 == True and c_ret2 == True:
                get.append([c1_images_names[i],c2_images_names[i]])
                corners1 = cv2.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
                corners2 = cv2.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)
    
                cv2.drawChessboardCorners(frame1, (rows, columns), corners1, c_ret1)
                
    
                cv2.drawChessboardCorners(frame2, (rows, columns), corners2, c_ret2)
                
                
                Sres = np.concatenate((frame1, frame2), axis=1)
                text = str(i+1) + ' | Camera 1 : ' + str(c1_images_names[i])
                cv2.putText(Sres, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (255,255,0), 2, cv2.LINE_AA)
                
                text = str(i+1) + ' | Camera 2 : ' + str(c2_images_names[i])
                cv2.putText(Sres, text, (width+50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (255,255,0), 2, cv2.LINE_AA)
                
                if show:
                    cv2.imshow('Sres', Sres)
                    if cv2.waitKey(0) & 0xFF == ord('q'):
                        show = False
                        cv2.destroyAllWindows()
                
    
                objpoints.append(objp)
                imgpoints_left.append(corners1)
                imgpoints_right.append(corners2)

            i += 1
                
        
        cv2.destroyAllWindows()
        stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC + cv2.CALIB_RATIONAL_MODEL

        ret, CM1, dist1, CM2, dist2, R, T, E, F = cv2.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx1, dist1,
                                                                    mtx2, dist2, (width, height), criteria = criteria, flags = stereocalibration_flags)

        yes_flags = cv2.CALIB_FIX_INTRINSIC + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_USE_EXTRINSIC_GUESS
        yes = cv2.stereoCalibrateExtended(objpoints, imgpoints_left, imgpoints_right, mtx1, dist1,
                                                                    mtx2, dist2, (width, height), R, T, criteria = criteria, flags = yes_flags)
        
        unideal_detected = False
        for j , error in enumerate(yes[-1]):
            if error[0] >= error_threshold or error[1] >= error_threshold:
                unideal_detected = True
                unideal_images[tuple(get[j])] = error
        
        if not unideal_detected:
            ideal = True
        

    print("RMSE :",ret)
    print('detected {n} :'.format(n=len(get)))     
    for k in range(len(get)):
        print("Iter :", k+1)
        print("1st Camera :",get[k][0],"Error :",yes[-1][k][0])
        print("2nd Camera :",get[k][1],"Error :",yes[-1][k][1])
        print()
        
    print("Unideal Images:")
    print(unideal_images)
        
    
    return R, T
    

# Stereo Folder

In [9]:
stereo_lab = 'k4'
stereo_folder = 'stereo'

# Do Calibrate

In [10]:
R1, T1 = stereo_calibrate(mtx_j, dist_j, mtx_m, dist_m, './data/'+stereo_lab+'/j/'+stereo_folder+'/*','./data/'+stereo_lab+'/m/'+stereo_folder+'/*',show=True, error_threshold=30)
R1, T1

RMSE : 0.2511967484237213
detected 6 :
Iter : 1
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1.jpg Error : 0.269865262775367
2nd Camera : ./data/k4/m/stereo\IMG_StereoDefault1.jpg Error : 0.2767960666270753

Iter : 2
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1_1.jpg Error : 0.31195786496031574
2nd Camera : ./data/k4/m/stereo\IMG_StereoDefault1_1.jpg Error : 0.3086208466460592

Iter : 3
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1_2.jpg Error : 0.32005815729444137
2nd Camera : ./data/k4/m/stereo\IMG_StereoDefault1_2.jpg Error : 0.3107204493196353

Iter : 4
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1_3.jpg Error : 0.24339553083923748
2nd Camera : ./data/k4/m/stereo\IMG_StereoDefault1_3.jpg Error : 0.20654277072773516

Iter : 5
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1_4.jpg Error : 0.1322887385654691
2nd Camera : ./data/k4/m/stereo\IMG_StereoDefault1_4.jpg Error : 0.14503585976439567

Iter : 6
1st Camera : ./data/k4/j/stereo\IMG_StereoDefault1_5.jpg Error

(array([[ 0.99337806, -0.05492961, -0.10090969],
        [ 0.05074094,  0.99775887, -0.04361885],
        [ 0.1030795 ,  0.03820975,  0.99393895]]),
 array([[19.38531458],
        [ 2.50027965],
        [ 2.69244287]]))

In [ ]:
R2, T2 = stereo_calibrate(mtx_j, dist_j, mtx_w, dist_w, './data/'+stereo_lab+'/j/'+stereo_folder+'/*','./data/'+stereo_lab+'/w/'+stereo_folder+'/*',show=True, error_threshold=30)
R2, T2